In [ ]:
import pandas as pd
import math 
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')
import utils_explore as ue

data = ue.all_bets()

In [89]:
# select interesting authors and the most recent time (~6 month)
authors = ['chaplygin', 'falcao1984', 'teplofevralya']
df = data[(data.author.isin(authors)) & (data['placed-date']>=datetime.date(2019, 5, 1))]

In [93]:
#group data by weeks, then add amount ratio for every author, 
# then calculate count and success rate with this new amounts
df_weekly = ue.calc_win_rate(df, 'W')
df_amount = pd.DataFrame({'author' : authors, 'ratio' : [2, 2, 1]}) # authors = ['chaplygin', 'falcao1984', 'teplofevralya']
df_weekly = df_weekly.merge(df_amount, on='author', how="left").drop(columns=['author','success_rate','return'])
df_weekly['count'] = df_weekly['count'] * df_weekly['ratio']
df_weekly['win'] = df_weekly['win'] * df_weekly['ratio']
#summarize on weeks
df_weekly = df_weekly.groupby(by='placed-date').agg('sum')
df_weekly['success_rate'] = df_weekly['win'] / df_weekly['count'] * 100
df_weekly.reset_index(inplace=True)
df_weekly

,placed-date,count,win,ratio,success_rate
0,2019-04-29/2019-05-05,26.0,17.11,5,65.807692
1,2019-05-06/2019-05-12,37.0,-3.02,5,-8.162162
2,2019-05-13/2019-05-19,30.0,0.76,5,2.533333
3,2019-05-20/2019-05-26,31.0,2.07,5,6.677419
4,2019-05-27/2019-06-02,33.0,4.25,5,12.878788
5,2019-06-03/2019-06-09,60.0,22.82,5,38.033333
6,2019-06-10/2019-06-16,31.0,4.21,5,13.580645
7,2019-06-17/2019-06-23,41.0,-4.32,5,-10.536585
8,2019-06-24/2019-06-30,20.0,5.82,4,29.100000
9,2019-07-01/2019-07-07,27.0,-1.26,5,-4.666667


In [94]:
# size of each stake depends on budget
# the budget spreads with respekt to amount ratio
budget = 800
single_stake = budget / df_weekly['count'].mean()
df_amount['stake'] = df_amount['ratio'] * single_stake
df_amount

,author,ratio,stake
0,chaplygin,2,47.346939
1,falcao1984,2,47.346939
2,teplofevralya,1,23.673469


In [95]:
#simple outcome 
df_weekly['fin-result'] = df_weekly['win'] * single_stake
df_weekly['fin-result'].sum()

2195.004081632653

In [96]:
tax = 0.95 #taxes
bets_inaccuracy = 0.9 #my odds are often differ (they are smaller) 
round(df_weekly['fin-result'].sum() * tax * bets_inaccuracy, 1)

1876.7